In [1]:
import torch
model = torch.hub.load("bryandlee/animegan2-pytorch", "generator", pretrained="paprika").eval()


/home/bowserj/anaconda3/lib/python3.9/site-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/bryandlee/animegan2-pytorch/zipball/main" to /home/bowserj/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/paprika.pt" to /home/bowserj/.cache/torch/hub/checkpoints/paprika.pt


  0%|          | 0.00/8.20M [00:00<?, ?B/s]

In [2]:
model

Generator(
  (block_a): Sequential(
    (0): ConvNormLReLU(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), bias=False)
      (2): GroupNorm(1, 32, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): ConvNormLReLU(
      (0): ReflectionPad2d((0, 1, 0, 1))
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (2): GroupNorm(1, 64, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): ConvNormLReLU(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (2): GroupNorm(1, 64, eps=1e-05, affine=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (block_b): Sequential(
    (0): ConvNormLReLU(
      (0): ReflectionPad2d((0, 1, 0, 1))
      (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (2): GroupNorm(1, 128, eps=1e-05, affi

In [3]:
from PIL import Image
import numpy as np

import torch
from torchvision.transforms.functional import to_tensor, to_pil_image

In [4]:
img = Image.open('packraft.png').convert("RGB")

In [5]:
image_input = to_tensor(img)

In [6]:
image_input = image_input * 2 - 1
image_input = image_input.unsqueeze(0)

In [7]:
im_out = model.forward(image_input)

In [8]:
out = im_out.squeeze(0).clip(-1, 1) * 0.5 + 0.5

In [9]:
out = to_pil_image(out)

In [10]:
out.show()

In [11]:
import torch.onnx
x = torch.randn(1, 3, 512, 512, requires_grad=True)
torch_out = model(x)

In [12]:
# Open ended Height and Width
# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "animegan_paprika.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=14,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size', 2: 'height', 3 : 'width'},    # variable length axes (required for batching)
                                'output' : {0 : 'batch_size', 2: 'height', 3 : 'width'}})

/home/bowserj/.cache/torch/hub/bryandlee_animegan2-pytorch_main/model.py:96: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if align_corners:
/home/bowserj/.cache/torch/hub/bryandlee_animegan2-pytorch_main/model.py:102: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if align_corners:
/home/bowserj/anaconda3/lib/python3.9/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /home/bowserj/pytorch/torch/c

In [13]:
# Open ended Height and Width
# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "animegan_paprika_constraned.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=14,          # the ONNX version to export the model to
                  do_constant_folding=True  # whether to execute constant folding for optimization
                  )

In [14]:
from torch.utils.mobile_optimizer import optimize_for_mobile
import torch.backends._nnapi.prepare

# Input to the model
dummy_input = torch.rand(1, 3, 512, 512)
torchscript_model = torch.jit.trace(model, dummy_input)
torchscript_model_optimized = optimize_for_mobile(torchscript_model)
torch.jit.save(torchscript_model_optimized, "animegan2.pt")

In [15]:
chan_last_model = model.to(memory_format=torch.channels_last)

In [16]:
#NHWC Model
dummy_input_last = torch.rand(1, 512, 512, 3)
traced_script_module = torch.jit.trace(chan_last_model, dummy_input)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
torch.jit.save(traced_script_module_optimized, "animegan2_nhwc.pt")

In [17]:
#Vulkan NCHW Backend
nchw_script_module_optimized = optimize_for_mobile(torchscript_model, backend='vulkan')
torch.jit.save(nchw_script_module_optimized, "animegan_vulkan_nchw.pt")

#Vulkan NHWC Backend
traced_script_module_vulkan_optimized = optimize_for_mobile(traced_script_module, backend='vulkan')
torch.jit.save(traced_script_module_vulkan_optimized, "animegan_vulkan_nhwc.pt")

MESA-INTEL: warning: Performance support disabled, consider sysctl dev.i915.perf_stream_paranoid=0

